2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Background:

Here, we learn how to create and use terraform maps. Maps are key/value pairs which we can create and look up as we need to in our templates.



<img align="left" src="../images/ThinBlueBar.png" width="400" /><br/>

## Tasks:
### 1. Make a directory called ‘lab4b’ underneath the labs directory.
### 2. Change into the directory.
### 3. Create the following files: main.tf, resources.tf vars.tf.

The main.tf file should contain:

The provider.tf file should contain:

In [4]:
cat provider.tf


terraform {
  required_version = ">= 1.1.0"

  required_providers {
    aws = {
      version = "~> 4.0"
    }
  }
}

provider "aws" {
  region = var.region

  default_tags {
    tags = {
      Environment = "Terraform Introduction"
    }
  }
}



The resources.tf file should contain:

In [5]:
cat resources.tf


resource "aws_vpc" "main_vpc" {
  cidr_block       = var.vpc_cidr
  instance_tenancy = "default"

  tags = {
    Name           = "Main"
    Location       = "London"
    LabName        = "4b.TerraformMaps"
  }
}

resource "aws_subnet" "vpc_subnets" {
  count = length(var.vpc_subnet_cidr)

  vpc_id             = aws_vpc.main_vpc.id

  #cidr_block        = element(var.vpc_subnet_cidr,count.index)
  cidr_block         = var.vpc_subnet_cidr[count.index]

  #availability_zone = element(var.aaz[var.region],count.index)
  availability_zone  = var.aaz[var.region][count.index]

  tags = {
    Name = "subnet-${count.index+1}"
    LabName = "4b.TerraformMaps"
  }
}


and your vars.tf file should contain:

In [6]:
cat vars.tf


variable "region" {
  default = "us-west-1"
}
variable "vpc_cidr" {
  default = "192.168.0.0/16"
}
variable "vpc_subnet_cidr" {
  type = list
  # Reduced to 2-elements each due to non-availability of us-west-1a:
  #default = ["192.168.100.0/24","192.168.101.0/24","192.168.102.0/24"]
  default = ["192.168.101.0/24","192.168.102.0/24"]
}
variable "ami_instance" {
  type = map
  default = {
    "us-east-1" = "ami-0ac019f4fcb7cb7e6"
    "us-east-2" = "ami-0f65671a86f061fcd"
    "us-west-1" = "ami-063aa838bd7631e0b"
  }
}
variable "ami_instance_type" {
  default = "t2.micro"
}
variable "aaz" {
  type = map
  default = {
    #"us-east-1" = ["us-east-1a","us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2a","us-east-2b","us-east-2c"]
    #"us-west-1" = ["us-west-1a","us-west-1b","us-west-1c"]

    # Reduced to 2-elements each due to non-availability of us-west-1a:
    #"us-east-1" = ["us-east-1b","us-east-1c"]
    #"us-east-2" = ["us-east-2b","us-east-2c"]
    "us-west-1" = ["us-west-1

You may copy the outputs.tf file you used for the previous lab to see the output.

<!-- data "aws_availability_zones" "aaz" {} -->

### 4. Study the configuration files

Study the configuration files, taking the time to understand

* the use of the "vpc_subnet_cidr" list variable

* the use of the "ami_instance" map variable

* the use of the "aaz" map variable

- why we need to modify the list/map dimensions when an availability zone is not available

### 5. The configuration when visualized should look like

<div>
    <object data="graph.svg" type="image/svg+xml">
    </object>
</div>

![](graph.svg)

### 6. Initialize the configuration

This time we will set the TF_LOG variable to TRACE to see debugging output: ```TF_LOG=TRACE terraform init```

In [7]:
TF_LOG=TRACE terraform init

2023-01-09T14:44:37.219Z [INFO]  Terraform version: 1.3.7
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/go-tfe v1.9.0
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/hcl/v2 v2.15.0
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/terraform-config-inspect v0.0.0-20210209133302-4fd17a0faac2
2023-01-09T14:44:37.219Z [DEBUG] using github.com/hashicorp/terraform-svchost v0.0.0-20200729002733-f050f53b9734
2023-01-09T14:44:37.219Z [DEBUG] using github.com/zclconf/go-cty v1.12.1
2023-01-09T14:44:37.219Z [INFO]  Go runtime version: go1.19.4
2023-01-09T14:44:37.219Z [INFO]  CLI args: []string{"terraform", "init"}
2023-01-09T14:44:37.219Z [TRACE] Stdout is a terminal of width 80
2023-01-09T14:44:37.219Z [TRACE] Stderr is a terminal of width 80
2023-01-09T14:44:37.219Z [TRACE] Stdin is a terminal
2023-01-09T14:44:37.219Z [DEBUG] Attempting to open CLI config file: /home/student/.terraformrc
2023-01-09T14:44:37.219Z [DEBUG] File doesn't exist, but doesn't need t

selections it made above. Include this file in your version control repository
so that Terraform can guarantee to make the same selections by default when
you run "terraform init" in the future.

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


This command initializes the terraform directory structure.

Take the time to look at the trace output to see what the *init* process does and where it looks for and stores provider plugin or module files.

This can be particularly useful when build or installing third-party provider plugins manually - to debug possible failure to find the appropriate binaries.

### 7.  Preview the configuration

In [8]:
terraform plan


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_subnet.vpc_subnets[0] will be created
  + resource "aws_subnet" "vpc_subnets" {
      + arn                                            = (known after apply)
      + assign_ipv6_address_on_creation                = false
      + availability_zone                              = "us-west-1b"
      + availability_zone_id                           = (known after apply)
      + cidr_block                                     = "192.168.101.0/24"
      + enable_dns64                                   = false
      + enable_resource_name_dns_a_record_on_launch    = false
      + enable_resource_name_dns_aaaa_record_on_launch = false
      + id                                             = (known after apply)
      + ipv6_cidr_block_association_id                 = (known after apply)
      +

**verify** the actions that terraform will take.

### 8. Apply the configuration

In [9]:
terraform apply 


Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # aws_subnet.vpc_subnets[0] will be created
  + resource "aws_subnet" "vpc_subnets" {
      + arn                                            = (known after apply)
      + assign_ipv6_address_on_creation                = false
      + availability_zone                              = "us-west-1b"
      + availability_zone_id                           = (known after apply)
      + cidr_block                                     = "192.168.101.0/24"
      + enable_dns64                                   = false
      + enable_resource_name_dns_a_record_on_launch    = false
      + enable_resource_name_dns_aaaa_record_on_launch = false
      + id                                             = (known after apply)
      + ipv6_cidr_block_association_id                 = (known after apply)
      +

Assuming that this works correctly, the AWS Provider will create a VPC, and 2 subnets located in 2 different
availability zones, and an ami instance running on each subnet.

### 9. Create some instances using the vpcs

#### Known error to be fixed ...

Getting error: ```Error launching source instance: InvalidParameter: Security group sg-07cbbd1042f3c81b3 and subnet subnet-2c6ba676 belong to different networks.```


Now add some new "aws_instance" resources, i.e. VMs, using count to create several.

Output the assigned public_ips of the VMs

Output also a combined output of public_ip, public_dns, private_ip for each VM of the form:
- "public address: <public_ip>[<public_dns>] private address: <private_ip>"

You should use the "*for*" expression to achieve this

#### There is an error in the above config - work in progress (regression)

We've achieved most of the lab, which is around the use of Terraform Maps

### 10. Investigate the *terraform state* sub-command

You can list the resources available in the current state using the ```terraform state list``` command.

Look at the state of one of the listed resources using command ```terraform state show <resource>```, e.g.
- ```terraform state show aws_vpc.main_vpc```
- ```terraform state show aws_instance.example[0]```

### 11. Cleanup

In [13]:
terraform destroy 

tls_private_key.mykey: Refreshing state... [id=e92e6d88710e7dd7c9bde48de0d8b282862feeef]
aws_key_pair.generated_key: Refreshing state... [id=LAB4b-key]
aws_vpc.main_vpc: Refreshing state... [id=vpc-0a27f3377fe36f60e]
aws_subnet.vpc_subnets[0]: Refreshing state... [id=subnet-063d030c0be1e1b6e]
aws_subnet.vpc_subnets[1]: Refreshing state... [id=subnet-0ff96eff4a1656bbd]
aws_security_group.secgroup-ssh: Refreshing state... [id=sg-049d574d73c3307d1]

Terraform used the selected providers to generate the following execution plan.
Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # aws_key_pair.generated_key will be destroyed
  - resource "aws_key_pair" "generated_key" {
      - arn         = "arn:aws:ec2:us-west-1:816376574968:key-pair/LAB4b-key" -> null
      - fingerprint = "6c:68:e1:3f:3d:5c:6f:85:29:56:68:74:e5:8a:76:0b" -> null
      - id          = "LAB4b-key" -> null
      - key_name    = "LAB4b-key" -> null
     

      - enable_dns64                                   = false -> null
      - enable_resource_name_dns_a_record_on_launch    = false -> null
      - enable_resource_name_dns_aaaa_record_on_launch = false -> null
      - id                                             = "subnet-0ff96eff4a1656bbd" -> null
      - ipv6_native                                    = false -> null
      - map_customer_owned_ip_on_launch                = false -> null
      - map_public_ip_on_launch                        = false -> null
      - owner_id                                       = "816376574968" -> null
      - private_dns_hostname_type_on_launch            = "ip-name" -> null
      - tags                                           = {
          - "LabName" = "4b.TerraformMaps"
          - "Name"    = "subnet-2"
        } -> null
      - tags_all                                       = {
          - "Environment" = "Terraform Introduction"
          - "LabName"     = "4b.TerraformMaps"
          - "

    EOT -> null
aws_security_group.secgroup-ssh: Destroying... [id=sg-049d574d73c3307d1]
aws_subnet.vpc_subnets[1]: Destroying... [id=subnet-0ff96eff4a1656bbd]
aws_subnet.vpc_subnets[0]: Destroying... [id=subnet-063d030c0be1e1b6e]
aws_key_pair.generated_key: Destroying... [id=LAB4b-key]
aws_key_pair.generated_key: Destruction complete after 1s
tls_private_key.mykey: Destroying... [id=e92e6d88710e7dd7c9bde48de0d8b282862feeef]
tls_private_key.mykey: Destruction complete after 0s
aws_subnet.vpc_subnets[1]: Destruction complete after 1s
aws_subnet.vpc_subnets[0]: Destruction complete after 1s
aws_security_group.secgroup-ssh: Destruction complete after 2s
aws_vpc.main_vpc: Destroying... [id=vpc-0a27f3377fe36f60e]
aws_vpc.main_vpc: Destruction complete after 1s

Destroy complete! Resources: 6 destroyed.



To destroy the formerly created AWS vpc, and all subnets.

<hr/>



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# Summary

In this Exercise we looked at the use of the *map* type.

We used this type to map from
- region to ami image
- region to availability zones

We then created some VM instances and output their subnet, zone information



<img align="left" src="../images/ThinBlueBar.png" /><br/>

# Solutions

Solutions are available in the *github* repo at ```https://github.com/mjbright/tf-scenarios``` under Solutions at https://github.com/mjbright/tf-scenarios/tree/main/Solutions/lab4b




<img align="left" src="../images/ThinBlueBar.png" /><br/>

# **Stretch Goal 1:** Dynamic Blocks

- Add a security group to your VMs, using a dynamic block to specify each ingress rule to allow incoming traffic on ports 22 and 8080
- Investigate the state of the dynamic block you created
- Verify that you can ssh into your instances

You might want to refer to https://github.com/hashicorp/terraform-guides/tree/master/infrastructure-as-code/terraform-0.12-examples/dynamic-blocks-and-splat-expressions for some hints

<hr/>
<!-- ![](../../../static/images/LOGO_v2_CROPPED.jpg)
<img src="../../../static/images/LOGO_v2_CROPPED.jpg" width="200" /> -->
<img src="../images/LOGO_v2_CROPPED.jpg" width="200" />

2023-Jan-09:[TF-1.3.7] Lab updated on node tf[terraform 1.3.7]


: 1